In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
PATH = '/kaggle/input/m5-forecasting-accuracy/'
cal = pd.read_csv(f'{PATH}calendar.csv')
# sell_prices = pd.read_csv(f'{PATH}sell_prices.csv')
ss = pd.read_csv(f'{PATH}sample_submission.csv')
stv = pd.read_csv(f'{PATH}sales_train_validation.csv')

Используем последние известные 28 дней, которые должны быть наиболее актуальными, т.к. были недавно. Рассчитаем спрос, сгруппированный по дням недели

In [ ]:
# Берем только последние 28 дней
last_28 = stv.iloc[:, pd.np.r_[0,-28:0]]
last_28.head()

In [ ]:
# Переместим дни в колонку 'd' и значения спроса в 'demand'
last = last_28.melt('id', var_name='d', value_name='demand')
last.head()

In [ ]:
# сделаем слияние с календарем
last = last.merge(cal)
last.head()

In [ ]:
# рассчитаем спрос по дню недели
by_weekday = last.groupby(['id','wday'])['demand'].mean()

In [ ]:
# скопируем пример
sub = ss.copy()
# поменяем названия колонок на соответствующие последним 28 дням
sub.columns = ['id'] + ['d_' + str(1914+x) for x in range(28)]
# возьмем строки со значением validation
sub = sub.loc[sub.id.str.contains('validation')]

In [ ]:
# разобьем таблицу и сделаем слияние с календарем
sub = sub.melt('id', var_name='d', value_name='demand')
sub = sub.merge(cal)[['id', 'd', 'wday']]
df = sub.join(by_weekday, on=['id', 'wday'])
df.head()

In [ ]:
# создадим структуру таблицы для сабмита
df = df.pivot(index='id', columns='d', values='demand')
# переназначим индекс
df.reset_index(inplace=True)
df.head()

In [ ]:
# Сделаем слияние данных для сабмита с id из df 
submission = ss[['id']].copy()
submission = submission.merge(df)

In [ ]:
# продублируем данные
submission = pd.concat([submission, submission], axis=0)

In [ ]:
# снова переназначим индексы для корректного сабмита
submission['id'] = ss.id.values

In [ ]:
# переименуем столбцы для корректного сабмита
submission.columns = ['id'] + ['F' + str(i) for i in range(1,29)]
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)